## Análise exploratória de dados

Visto que o site foi feito em [Shiny](https://shiny.rstudio.com/), a os gráficos desta seção foram feitos utilizando os pacotes ggplot2 e plotly do R, abaixo seguem guias de como utilizar o R e o plotly em um notebook Jupyter.

Guia para utilizar o R no Jupyter: https://docs.anaconda.com/anaconda/navigator/tutorials/r-lang/

Guia para usar o Plotly no Jupyter: https://plot.ly/r/using-r-in-jupyter-notebooks/

Pacotes necessários:

In [ ]:
# executar a próxima linha comentada para instalar os pacotes
# install.packages(c("dplyr", "stringr", "ggplot2", "plotly"))

library(dplyr)
library(stringr)
library(ggplot2)
library(plotly)

In [ ]:
options(encoding = "UTF-8")
options(OutDec = ",")

tema = theme(
  panel.grid.minor = element_blank(), 
  panel.grid.major = element_blank(),
  panel.background = element_blank(),
  plot.background = element_blank(),
  legend.background = element_blank(),
  legend.key = element_blank(),
  text = element_text(size = 10)
)

paises = c("Argentina", "Bolívia", "Brasil", "Chile", "Colômbia", "Equador", "Paraguai", "Peru", "Uruguai", "Venezuela")

## Partidas por confederação e vantagem do mandante

In [ ]:
files = c("data/jogosarg.csv", "data/jogosbol.csv", "data/jogosbra.csv", "data/jogoschi.csv", "data/jogoscol.csv",
          "data/jogosecu.csv", "data/jogospar.csv", "data/jogosper.csv", "data/jogosuru.csv", "data/jogosven.csv",
          "data/jogossula.csv", "data/jogosid.csv")

conf = c(paises, "Conmebol", "Todas")

df_mandante = data.frame()
df_partidas = data.frame()

for(i in 1:length(files)) {
  jogos = read.csv(files[i]) %>%
    mutate(resultado = if_else(g1 > g2, "Mandante",
                               if_else(g1 == g2, "Empate", "Visitante")))
  n = nrow(jogos)
  c = count(jogos, resultado)
  mandante = c$n[which(c$resultado == "Mandante")]
  empate = c$n[which(c$resultado == "Empate")]
  visitante = c$n[which(c$resultado == "Visitante")]
  
  tmp = tibble(Confederacao = c(conf[i], conf[i], conf[i]),
               Vencedor = c("Mandante", "Empate", "Visitante"),
               Freq = c(100*mandante/n, 100*empate/n, 100*visitante/n),
               n = c(mandante, empate, visitante))
  
  df_mandante = df_mandante %>%
    rbind(tmp)
  
  tmp2 = tibble(Confederacao = conf[i],
                Partidas = n)
  
  df_partidas = df_partidas %>%
    rbind(tmp2)
}

ordem_conf = c("Todas", "Conmebol", paises)
ordem_vencedor = c("Mandante", "Empate", "Visitante")

df_mandante = df_mandante %>%
  mutate(Confederacao = factor(Confederacao, levels = ordem_conf),
         Vencedor = factor(Vencedor, levels = ordem_vencedor),
         text = paste0(n, " (", round(Freq, 2), "%)"))

total = max(df_partidas$Partidas)
  
df_partidas = df_partidas %>%
  mutate(Confederacao = factor(Confederacao, levels = ordem_conf),
         porc = round(100*Partidas/total, 2),
         text = paste0(Partidas, " (", porc, "%)"))

In [ ]:
p = df_partidas %>%
  rename(Confederação = Confederacao) %>%
  ggplot(aes(x = Confederação, y = Partidas, text = text)) +
  geom_bar(stat = "identity", position = "dodge", fill = "#F8766D") +
  tema +
  xlab("") +
  ylab("Partidas") +
  ggtitle("Partidas por confederação")

p %>%
  ggplotly(tooltip = c("text")) %>%
  layout(plot_bgcolor = 'rgba(0, 0, 0, 0)',
         paper_bgcolor = 'rgba(0, 0, 0, 0)')

Neste gráfico pode-se observar uma vantagem para as principais nações dentro do cenário do futebol sul-americano, com destaque para a Colômbia com 7888 jogos. 

In [ ]:
p = df_mandante %>%
  ggplot(aes(x = Confederacao, y = Freq, fill = Vencedor, text = text)) +
  geom_bar(stat = "identity", position = "dodge") +
  tema +
  xlab("") +
  ylab("Resultado (%)") +
  scale_y_continuous(breaks = c(10, 20, 30, 40, 50)) +
  ggtitle("Influência do mando de campo por confederação")

p %>%
  ggplotly(tooltip = c("text")) %>%
  layout(plot_bgcolor = 'rgba(0, 0, 0, 0)',
         paper_bgcolor = 'rgba(0, 0, 0, 0)')

A influência do mando de campo no futebol sempre foi alvo de discussões entre especialistas e amantes do esporte. Neste gráfico estão dispostos os resultados das partidas considerando o mando de campo. Em todas as federações este fator vem se mostrando decisiva com peso de até 56,68%, como é o caso da Bolívia.

## Placares mais comuns

In [ ]:
tmp = read.csv("data/jogosid.csv") %>%
  mutate(Placar = paste(g1, "x", g2)) %>%
  count(Placar) %>%
  arrange(desc(n))

placares = tmp %>%
  head(20)

outros = sum(tmp$n) - sum(placares$n)

total = sum(tmp$n)

placares = placares %>%
  rbind(data.frame(Placar = "Outros", n = outros)) %>%
  mutate(Placar = factor(Placar, levels = Placar),
         porc = round(100*n/total, 2),
         text = paste0(n, " (", porc, "%)"))

In [ ]:
p = placares %>%
  ggplot(aes(x = Placar, y = n, text = text)) +
  geom_bar(stat = "identity", position = "dodge", fill = "#F8766D") +
  tema +
  xlab("") +
  ylab("Quantidade") +
  ggtitle("Placares mais comuns")

p %>%
  ggplotly(tooltip = c("text")) %>%
  layout(plot_bgcolor = 'rgba(0, 0, 0, 0)',
         paper_bgcolor = 'rgba(0, 0, 0, 0)')

Vemos que na base de dados, os quatro placares mais comuns são de empates ou vitórias do time mandante por uma margem de apenas um gol.

## Confrontos internacionais

In [ ]:
sula = read.csv("data/jogossulaid.csv") %>%
  select(-X, -date, -tourn) %>%
  mutate(p1 = NA, p2 = NA) %>%
  mutate(resultado = if_else(g1 > g2, "vit",
                          if_else(g1 == g2, "emp", "der")))
  
sula$p1[which(sula$id1 > 0 & sula$id1 <= 100)] = "Argentina"
sula$p1[which(sula$id1 > 100 & sula$id1 <= 200)] = "Bolívia"
sula$p1[which(sula$id1 > 200 & sula$id1 <= 300)] = "Brasil"
sula$p1[which(sula$id1 > 300 & sula$id1 <= 400)] = "Chile"
sula$p1[which(sula$id1 > 400 & sula$id1 <= 500)] = "Colômbia"
sula$p1[which(sula$id1 > 500 & sula$id1 <= 600)] = "Equador"
sula$p1[which(sula$id1 > 600 & sula$id1 <= 700)] = "Paraguai"
sula$p1[which(sula$id1 > 700 & sula$id1 <= 800)] = "Peru"
sula$p1[which(sula$id1 > 800 & sula$id1 <= 900)] = "Uruguai"
sula$p1[which(sula$id1 > 900 & sula$id1 <= 1000)] = "Venezuela"

sula$p2[which(sula$id2 > 0 & sula$id2 <= 100)] = "Argentina"
sula$p2[which(sula$id2 > 100 & sula$id2 <= 200)] = "Bolívia"
sula$p2[which(sula$id2 > 200 & sula$id2 <= 300)] = "Brasil"
sula$p2[which(sula$id2 > 300 & sula$id2 <= 400)] = "Chile"
sula$p2[which(sula$id2 > 400 & sula$id2 <= 500)] = "Colômbia"
sula$p2[which(sula$id2 > 500 & sula$id2 <= 600)] = "Equador"
sula$p2[which(sula$id2 > 600 & sula$id2 <= 700)] = "Paraguai"
sula$p2[which(sula$id2 > 700 & sula$id2 <= 800)] = "Peru"
sula$p2[which(sula$id2 > 800 & sula$id2 <= 900)] = "Uruguai"
sula$p2[which(sula$id2 > 900 & sula$id2 <= 1000)] = "Venezuela"

mat_confrontos = matrix(0, nrow = 10, ncol = 10)
colnames(mat_confrontos) = paises
rownames(mat_confrontos) = paises

for(i in 1:10) {
  for(j in 1:10) {
    if(i!=j) {
      
      vit = 0
      emp = 0
      
      tmp1 = sula %>%
        filter(p1 == rownames(mat_confrontos)[i],
               p2 == colnames(mat_confrontos)[j])
      
      c1 = count(tmp1, resultado)
      
      n1 = nrow(tmp1)
      
      tmp_vit = c1 %>% 
        filter(resultado == "vit") %>% 
        .$n
      
      if(length(tmp_vit) > 0) {
        vit = vit + tmp_vit
      }
      
      tmp_emp = c1 %>% 
        filter(resultado == "emp") %>% 
        .$n
      
      if(length(tmp_emp) > 0) {
        emp = emp + tmp_emp
      }
      
      tmp2 = sula %>%
        filter(p2 == rownames(mat_confrontos)[i],
               p1 == colnames(mat_confrontos)[j])
      
      c2 = count(tmp2, resultado)
      
      n2 = nrow(tmp2)
      
      tmp_vit = c2 %>% 
        filter(resultado == "der") %>% 
        .$n
      
      if(length(tmp_vit) > 0) {
        vit = vit + tmp_vit
      }
      
      tmp_emp = c2 %>% 
        filter(resultado == "emp") %>% 
        .$n
      
      if(length(tmp_emp) > 0) {
        emp = emp + tmp_emp
      }
      
      n = n1 + n2
      
      mat_confrontos[i, j] = (2*vit + emp) / (n*2)
      
    }
  }
}

diag(mat_confrontos) = NA

df_confrontos = tibble()

for(i in 1:10) {
  for(j in 1:10) {
    tmp = tibble(Origem = rownames(mat_confrontos)[i],
                 Adversario = colnames(mat_confrontos)[j],
                 Aproveitamento = mat_confrontos[j,i])
    df_confrontos = df_confrontos %>%
      rbind(tmp)
  }
}

df_confrontos = df_confrontos %>%
  mutate(Aproveitamento = 100*Aproveitamento,
         text = paste0(as.character(round(Aproveitamento, 2)), "%"))

df_confrontos$text[is.na(df_confrontos$Aproveitamento)] = ""

vitorias = NULL
empates = NULL
partidas = NULL

for(i in 1:length(paises)) {
  
  vit = 0
  emp = 0
  
  tmp1 = sula %>%
    filter(p1 == paises[i], p2 != paises[i])
  
  c1 = count(tmp1, resultado)
  
  n1 = nrow(tmp1)
  
  tmp_vit = c1 %>% 
    filter(resultado == "vit") %>% 
    .$n
  
  if(length(tmp_vit) > 0) {
    vit = vit + tmp_vit
  }
  
  tmp_emp = c1 %>% 
    filter(resultado == "emp") %>% 
    .$n
  
  if(length(tmp_emp) > 0) {
    emp = emp + tmp_emp
  }
  
  tmp2 = sula %>%
    filter(p2 == paises[i], p1 != paises[i])
  
  c2 = count(tmp2, resultado)
  
  n2 = nrow(tmp2)
  
  tmp_vit = c2 %>% 
    filter(resultado == "der") %>% 
    .$n
  
  if(length(tmp_vit) > 0) {
    vit = vit + tmp_vit
  }
  
  tmp_emp = c2 %>% 
    filter(resultado == "emp") %>% 
    .$n
  
  if(length(tmp_emp) > 0) {
    emp = emp + tmp_emp
  }
  
  vitorias[i] = vit
  empates[i] = emp
  partidas[i] = n1 + n2
  
}

confrontos = tibble(Pais = paises, Vitorias = vitorias, Empates = empates, Partidas = partidas) %>%
  mutate(Aproveitamento = 100*(2*Vitorias + Empates) / (Partidas*2),
         text = paste0("País: ", Pais, "<br>",
                       "Aproveitamento: ", round(Aproveitamento, 2), "%<br>",
                       "Vitórias: ", Vitorias, "<br>",
                       "Empates: ", Empates, "<br>",
                       "Partidas: ", Partidas, "<br>")) %>%
  arrange(desc(Aproveitamento)) %>%
  mutate(Pais = factor(Pais, levels = Pais))

In [ ]:
plot_ly(x = df_confrontos$Origem, y = df_confrontos$Adversario, z = df_confrontos$Aproveitamento, type = "heatmap", colorscale = "RdBu",
            reversescale = TRUE, hoverinfo = "none") %>%
      layout(plot_bgcolor = 'rgba(0, 0, 0, 0)',
             paper_bgcolor = 'rgba(0, 0, 0, 0)',
             xaxis = list(title = FALSE, fixedrange = TRUE),
             yaxis = list(title = FALSE, fixedrange = TRUE),
             title = "Aproveitamento em confrontos internacionais") %>% 
      add_annotations(x = df_confrontos$Origem, y = df_confrontos$Adversario, text = df_confrontos$text,
                      showarrow = FALSE, font = list(color = 'black'))

In [ ]:
p = confrontos %>%
  rename(País = Pais) %>%
  ggplot(aes(x = País, y = Aproveitamento, text = text)) +
  geom_bar(stat = "identity", position = "dodge", fill = "#F8766D") +
  tema +
  xlab("") +
  ylab("Aproveitamento (%)") +
  ggtitle("Aproveitamento em confrontos internacionais")

p %>%
  ggplotly(tooltip = c("text")) %>%
  layout(plot_bgcolor = 'rgba(0, 0, 0, 0)',
         paper_bgcolor = 'rgba(0, 0, 0, 0)')

Podemos ver que o Brasil é o país com melhor aproveitamento nos confrontos internacionais; os clubes brasileiros apresentam um equilíbro em confrontos com equipes da Argentina e levam vantagem contra clubes dos demais países. 
Acima do 50% de aproveitamento, além do Brasil, temos a Argentina, Colômbia e Equador. Em contrapartida, as equipes da Venezuela são as que demonstram o pior desempenho nos confrontos internacionais com 30,89% no geral e apenas 10,83% em confrontos contra clubes brasileiros.